In [ ]:
import pandas as pd
import ipywidgets as wg

from plotly import graph_objs as plgo

In [ ]:
data = pd.read_csv('pokemons.csv')
data.id = data.id.astype(int)
data.height = data.height.astype(int)
data.weight = data.weight.astype(int)
data.base_experience = data.base_experience.astype(int)

# Plotly
---

In [ ]:
data.head()

## First plot
---

In [ ]:
exp_frame = data.groupby('type1').base_experience.mean().to_frame()
exp_frame['std_dev'] = data.groupby('type1').base_experience.std()
exp_frame['n_main'] = data.groupby('type1').id.count()
exp_frame['n_sec'] = data.groupby('type2').id.count()
exp_frame = exp_frame.sort_values('base_experience')
exp_frame.head()

In [ ]:
fig = plgo.FigureWidget()

main_trace = plgo.Bar()
main_trace.x = exp_frame.index
main_trace.y = exp_frame.n_main
main_trace.name = 'Main'
main_trace.marker.color = 'rgb(25, 111, 61)'
main_trace.yaxis = 'y1'
fig.add_trace(main_trace)

sec_trace = plgo.Bar()
sec_trace.x = exp_frame.index
sec_trace.y = exp_frame.n_sec
sec_trace.name = 'Secondary'
sec_trace.marker.color = 'rgb(34, 153, 84)'
sec_trace.yaxis = 'y1'
fig.add_trace(sec_trace)

exp_trace = plgo.Scatter()
# exp_trace.mode = 'markers'
exp_trace.x = exp_frame.index
exp_trace.y = exp_frame.base_experience
exp_trace.name = 'Avg base exp.'
exp_trace.marker.color = 'rgb(111, 25, 75)'
exp_trace.yaxis = 'y2'

# exp_trace.error_y.array = exp_frame.std_dev
# exp_trace.error_y.color = 'rgb(111, 25, 75)'

fig.add_trace(exp_trace)

fig.layout.title = 'Pokemon Types and Subtypes'
fig.layout.xaxis.title = 'Type'
fig.layout.xaxis.domain = (0, 0.98)
# fig.layout.xaxis.rangeslider.visible = True

fig.layout.yaxis.title = 'N° of pokemons'
fig.layout.barmode = 'stack'


fig.layout['yaxis2'] = plgo.layout.YAxis()
fig.layout.yaxis2.title = 'Experience'
fig.layout.yaxis2.side = 'right'
fig.layout.yaxis2.overlaying = 'y1'
fig.layout.yaxis2.rangemode = 'tozero'

fig.layout.titlefont.size = 30

fig

## Second plot
---

In [ ]:
color_lookup = {
    'bug': '#FFD54F',
    'poison': '#512DA8',
    'grass': '#43A047',
    'water': '#1976D2',
    'electric': '#FFEE58',
    'rock': '#795548',
    'normal': '#FFE0B2',
    'fire': '#FF5722',
    'ghost': '#F3E5F5',
    'fairy': '#FFCDD2',
    'ground': '#BCAAA4',
    'flying': '#F0F4C3',
    'ice': '#B2EBF2',
    'steel': '#B0BEC5',
    'dark': '#000000',
    'psychic': '#BA68C8',
    'fighting': '#B71C1C',
    'dragon': '#004D40',
}

In [ ]:
fig2 = plgo.FigureWidget()

for elem, group in data.groupby('type1'):
    trace = plgo.Scatter()
    trace.name = elem
    trace.x = group.weight
    trace.y = group.height
    trace.mode = 'markers'
    
    trace.marker.color = color_lookup[elem]
    trace.marker.size = group.base_experience / 17
    trace.marker.line.width = 0
    
    trace.hoverinfo = 'text+name'
    trace.text = 'N°: ' + group.id.astype(str) + '<br>Name: ' + group.name
    
    fig2.add_trace(trace)
    
fig2.layout.title = 'Height vs Weight'
fig2.layout.yaxis.title = 'Height [m]'
fig2.layout.xaxis.title = 'Weight [Kg]'

fig2.layout.hovermode = 'closest'
fig2.layout.xaxis.type = 'log'
fig2.layout.yaxis.type = 'log'

fig2

## Widget Integration !!
---

In [ ]:
# Widget 1

slider = wg.IntSlider(
    value = 17,
    min = 2,
    max = 30,
    step = 1,
    description = 'Scale:'
)

def change_scale(change):
    with fig2.batch_animate():
        for i, (elem, group) in enumerate(data.groupby('type1')):
            fig2.data[i].marker.size = group.base_experience / change['new']
            
slider.observe(change_scale, names='value')

# Widget 2

text = wg.Text(
    placeholder = 'Type a title for the plot.',
    description = 'Title:'
)

def change_title(change):
    with fig2.batch_update():
        fig2.layout.title = change['new']
        
text.observe(change_title, names='value')

# Widget 3

flying_color = wg.ColorPicker(
    description = 'Flying type:',
    value = '#F0F4C3'
)

def change_color(change):
    with fig2.batch_animate():
        for i, (elem, group) in enumerate(data.groupby('type1')):
            if fig2.data[i].name == 'flying':
                fig2.data[i].marker.color = change['new']

flying_color.observe(change_color, names='value')

# Widget 4

image = wg.HTML()

def change_image(trace, points, input_device_state):
    if len(points.point_inds) != 0:
        poke = data[data.type1 == points.trace_name].iloc[points.point_inds[0]]
        image.value = '<img src="{}">'.format(
            poke.image_url_shiny if input_device_state.ctrl else poke.image_url
        )
    
for trace in fig2.data:
    trace.on_hover(change_image)
    
# Display

wg.VBox([
    fig2,
    wg.HBox([
        wg.VBox([text, slider, flying_color]),
        image
    ])
])

## Figures as widgets
---

In [ ]:
type_series = exp_frame.reset_index().type1

In [ ]:
def link_to_fig1(trace, points, input_device_state):
    if len(points.point_inds) != 0:
        with fig.batch_update():
            for trace in fig.data:
                trace.marker.opacity = type_series.map(
                    lambda x: 1 if x == points.trace_name else 0.2
                )
            fig.data[2].line.color = 'rgba(111, 25, 75, 0.2)'

for trace in fig2.data:
    trace.on_hover(link_to_fig1)
    
def link_to_fig2(trace, points, input_device_state):
    if len(points.point_inds) != 0:
        with fig2.batch_update():
            for t in fig2.data:
                t.visible = True if t.name == points.xs[0] else 'legendonly'
                
fig.data[0].on_click(link_to_fig2)

def autolink_fig1(trace, points, input_device_state):
    if len(points.point_inds) != 0:
        with fig.batch_update():
            for trace in fig.data:
                trace.marker.opacity = type_series.map(
                    lambda x: 1 if x == points.xs[0] else 0.2
                )
            fig.data[2].line.color = 'rgba(111, 25, 75, 0.2)'
            
fig.data[0].on_hover(autolink_fig1)

wg.VBox([fig, fig2])